In [6]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.linear_model import LinearRegression

In [17]:
df1 = pd.read_csv('/workspaces/EDaC-1/homework_3.1.csv')
df_a = pd.read_csv('/workspaces/EDaC-1/homework_3.2.a.csv')
df_b = pd.read_csv('/workspaces/EDaC-1/homework_3.2.b.csv')

In [ ]:
def estimate_level_jump(df, time_col, outcome_col, cutoff=50):
    # Build RDD design: running, level jump D, slope change running_D
    df = df.copy()
    df['running']   = df[time_col] - cutoff
    df['D']         = (df[time_col] >= cutoff).astype(int)
    df['running_D'] = df['running'] * df['D']
    
    X = df[['running', 'D', 'running_D']]
    y = df[outcome_col]
    
    model = LinearRegression().fit(X, y)
    # coef_[1] corresponds to the jump at the cutoff (the D indicator)
    return model.coef_[1]


# estimate level‐discontinuity for each series
for col in ['value1', 'value2', 'value3']:
    jump = estimate_level_jump(df1, 'time', col, cutoff=50)
    print(f"{col}: estimated jump at t=50 = {jump:.3f}")

value1: estimated jump at t=50 = 0.903
value2: estimated jump at t=50 = 0.701
value3: estimated jump at t=50 = 1.793


In [16]:
import pandas as pd
from sklearn.linear_model import LinearRegression

def estimate_slope_jump(df, time_col, outcome_col, cutoff=50):
    df = df.copy()
    # 1) Centered running variable
    df['running']   = df[time_col] - cutoff
    # 2) Post‐cut indicator
    df['D']         = (df[time_col] >= cutoff).astype(int)
    # 3) Interaction term for slope change
    df['running_D'] = df['running'] * df['D']
    
    X = df[['running', 'D', 'running_D']]
    y = df[outcome_col]
    
    model = LinearRegression().fit(X, y)
    slope_before = model.coef_[0]
    slope_jump   = model.coef_[2]
    slope_after  = slope_before + slope_jump
    
    return slope_before, slope_after, slope_jump

# load the dataset


# loop through each value*
for ycol in ['value1','value2','value3']:
    sl, sr, jump = estimate_slope_jump(df1, 'time', ycol, cutoff=50)
    print(f"{ycol:7s} | slope_before = {sl: .3f}  "
          f"slope_after = {sr: .3f}  "
          f"slope_jump = {jump: .3f}")

value1  | slope_before = -0.009  slope_after =  0.097  slope_jump =  0.105
value2  | slope_before =  0.010  slope_after =  0.047  slope_jump =  0.037
value3  | slope_before =  0.011  slope_after =  0.062  slope_jump =  0.051


In [18]:
def compute_did(df, group_col, time_col, outcome_col):
    # Average outcomes by group and period
    pre_treated   = df[(df[group_col] == 1) & (df[time_col] == 0)][outcome_col].mean()
    pre_control   = df[(df[group_col] == 0) & (df[time_col] == 0)][outcome_col].mean()
    post_treated  = df[(df[group_col] == 1) & (df[time_col] == 1)][outcome_col].mean()
    post_control  = df[(df[group_col] == 0) & (df[time_col] == 1)][outcome_col].mean()
    
    # Differences
    diff_pre  = pre_treated  - pre_control
    diff_post = post_treated - post_control
    
    # DiD estimate
    did = diff_post - diff_pre
    
    return {
        'pre_treated':   pre_treated,
        'pre_control':   pre_control,
        'post_treated':  post_treated,
        'post_control':  post_control,
        'diff_pre':      diff_pre,
        'diff_post':     diff_post,
        'did':           did
    }


# Compute DiD for each
res_a = compute_did(df_a, 'group1', 'time1', 'outcome1')
res_b = compute_did(df_b, 'group2', 'time2', 'outcome2')

print("Group 1 DiD:", res_a['did'])
print("Group 2 DiD:", res_b['did'])

Group 1 DiD: 0.6858469689928857
Group 2 DiD: 1.349858924693997


In [20]:
def did_with_se(df, group_col, time_col, outcome_col):
    # Prepare interaction term
    df = df.copy()
    df['interaction'] = df[group_col] * df[time_col]
    
    # Build design matrix [Intercept, Group, Time, Interaction]
    X = df[[group_col, time_col, 'interaction']].to_numpy()
    X = np.column_stack([np.ones(len(df)), X])
    y = df[outcome_col].to_numpy()
    
    # OLS estimates: beta = (X'X)^{-1} X'y
    XtX = X.T @ X
    XtX_inv = np.linalg.inv(XtX)
    beta = XtX_inv @ X.T @ y
    
    # Residual variance σ² = (e'e) / (n−k)
    resid = y - X @ beta
    n, k = X.shape
    sigma2 = resid.dot(resid) / (n - k)
    
    # Var–Cov matrix of β: σ² (X'X)^{-1}
    cov_beta = sigma2 * XtX_inv
    se_beta = np.sqrt(np.diag(cov_beta))
    
    # DiD estimate is β₃ (interaction term)
    did  = beta[3]
    se   = se_beta[3]
    tstat = did / se
    return did, se, tstat


# Compute DiD estimate, its SE, and t‐statistic for each
for name, df, gcol, tcol, ycol in [
    ('Group 1', df_a, 'group1', 'time1', 'outcome1'),
    ('Group 2', df_b, 'group2', 'time2', 'outcome2'),
]:
    did, se, tstat = did_with_se(df, gcol, tcol, ycol)
    print(f"{name}: DiD = {did:.3f}, SE = {se:.3f}, t = {tstat:.2f}")

Group 1: DiD = 0.686, SE = 0.063, t = 10.97
Group 2: DiD = 1.350, SE = 0.147, t = 9.18
